https://youtu.be/VmQLG-wJ624

📌 第一部分：ESP32 連接 Wi-Fi       

📢 ESP32 需要先連上 Wi-Fi，才能讀取天氣資料

In [ ]:
import network
import time

# 1️ Wi-Fi 資訊
SSID = "Wi-fi name"
PASSWORD = "Wi-fi passeord"

wifi = network.WLAN(network.STA_IF)
wifi.active(True)
wifi.connect(SSID, PASSWORD)

while not wifi.isconnected():
    print("正在連接 Wi-Fi...")
    time.sleep(1)

print("Wi-Fi 連線成功！")

📌 第二部分：ESP32 讀取 OpenWeather API     

📢 ESP32 透過 OpenWeather API 取得當地天氣

In [ ]:
import network
import urequests
import time
import ujson as json

# 1️ Wi-Fi 資訊
SSID = "Wi-fi name"
PASSWORD = "Wi-fi passeord"

# 2️ OpenWeather API 設定
WEATHER_URL = "Open Weather API Call"

# 連接 Wi-Fi
wifi = network.WLAN(network.STA_IF)
wifi.active(True)
wifi.connect(SSID, PASSWORD)

while not wifi.isconnected():
    print("正在連接 Wi-Fi...")
    time.sleep(1)

print("Wi-Fi 連線成功！")

# 讀取天氣資料
response = urequests.get(WEATHER_URL)

print(response.text)

📌 第三部分：ESP32 上傳天氣資料到 Google Sheets     

📢 ESP32 透過 Google Apps Script 把天氣資料送到雲端試算表

In [ ]:
import network
import urequests
import time
import ujson as json

# 1️ Wi-Fi 資訊
SSID = "Wi-fi name"
PASSWORD = "Wi-fi passeord"

# 2️ OpenWeather API 設定
WEATHER_URL = "Open Weather API Call"

# 3️ Google Apps Script Web App
WEB_APP_URL = "Google Apps Script Web API"

# 連接 Wi-Fi
wifi = network.WLAN(network.STA_IF)
wifi.active(True)
wifi.connect(SSID, PASSWORD)
while not wifi.isconnected():
    print("正在連接 Wi-Fi...")
    time.sleep(1)
print("Wi-Fi 連線成功！")

while True:
    # 讀取天氣資料
    response = urequests.get(WEATHER_URL)
    weather_data = json.loads(response.text)
    Kelvin_temp  = weather_data["main"]["temp"]
    celsius_temp = Kelvin_temp - 273.15  # 開爾文轉攝氏
    humidity = weather_data["main"]["humidity"]
    description = weather_data["weather"][0]["description"]
    response.close()
    # 準備上傳 Google Sheets 的資料
    data = {
        "city": "Taipei",
        "temperature": celsius_temp,
        "humidity": humidity,
        "description": description
    }

    # 發送請求到 Google Sheets
    res = urequests.post(WEB_APP_URL, json=data)
    print("已將天氣資訊上傳至 Google Sheets！", res.text)
    res.close()

    # 等待5秒鐘
    time.sleep(5)

📌 第四部分：Google Apps Script 處理資料        

📢 這段 Apps Script 會收到 ESP32 傳來的資料，並存入 Google Sheets

In [ ]:
# Google Apps Script 處理收到的資料 (Google Apps Script to handle received data)

function doPost(e) {
  var ss = SpreadsheetApp.openByUrl("你的Google sheet網址");
  var sheet = ss.getSheetByName("工作表名稱");
  var data = JSON.parse(e.postData.contents); // 解析收到的 JSON 資料
  var lastRow = sheet.getLastRow(); // 取得最後一行的行數
 
  // 如果行數超過 100，則從第 2 行開始覆蓋
  var startRow = (lastRow >= 100) ? 2 : lastRow + 1;

  // 在指定行插入數據，保留第一行
  sheet.getRange(startRow, 1, 1, 5).setValues([
    [new Date(), data.city, data.temperature, data.humidity, data.description]
  ]);
  return ContentService.createTextOutput("OK");
}